In [76]:
import weaviate
from weaviate.embedded import EmbeddedOptions

In [77]:
client = weaviate.Client(
    embedded_options=EmbeddedOptions(
        additional_env_vars={
            "ENABLE_MODULES": "backup-s3,text2vec-openai,text2vec-cohere,text2vec-huggingface,ref2vec-centroid,generative-openai,qna-openai"
        }
    )
)

embedded weaviate is already listening on port 8079


In [78]:
questions = client.data_object.get_by_id("c123e54e-0174-49e1-a318-96fedea484cf", class_name="Chatpdf")
questions

In [79]:
client.query.get(
    class_name="Chatpdf", properties=["chunk", "file_name"]
).with_near_text({"concepts": ["history of git"]}).with_limit(3).do()

{"action":"requests_total","api":"graphql","class_name":"","error":"Unknown argument \"nearText\" on field \"Chatpdf\" of type \"GetObjectsObj\". Did you mean \"nearVector\" or \"nearObject\"?","level":"error","msg":"unexpected error","query_type":"","time":"2023-12-18T01:38:46-05:00"}


{'errors': [{'locations': [{'column': 23, 'line': 1}],
   'message': 'Unknown argument "nearText" on field "Chatpdf" of type "GetObjectsObj". Did you mean "nearVector" or "nearObject"?',
   'path': None}]}